In [1]:
# train_PyTDC_new -> 42464 rows = 33028 rows + 9436 rows
# test_PyTDC_new -> 4718 rows

# PyTDC Rows -> epitope_aa, epitope_smi, tcr, tcr_full, label
# TLIGDCATV,
# CC[C@H](C)[C@H](NC(=O)[C@H](CC(C)C)NC(=O)[C@@H](N)[C@@H](C)O)C(=O)NCC(=O)N[C@@H](CC(=O)O)C(=O)N[C@@H](CS)C(=O)N[C@@H](C)C(=O)N[C@H](C(=O)N[C@H](C(=O)O)C(C)C)[C@@H](C)O,
# CASSEWGMDGTTDTQYF,
# NAGVTQTPKFQVLKTGQSMTLQCAQDMNHNSMYWYRQDPGMGLRLIYYSASEGTTDKGEVPNGYNVSRLNKREFSLRLESAAPSQTSVYFCASSEWGMDGTTDTQYFGPGTRLTVL,
# 1

# AVIB Beta Rows -> tcra, tcrb, peptide, sign
# unknown
# CASRECIATWHHQPQHF
# SSLENFRAYV
# 0-1

# tcra to unkown
# our tcr -> tcrb
# our epitope_aa -> peptide
# our label -> sign

# Step 1: Install necessary libraries (if not already installed)
import pandas as pd

# Step 2: Clone the GitHub repository to access the dataset
!git clone https://github.com/Yichuan0712/11785-TCR.git

# Step 3: Set the file paths
base_path = "/content/11785-TCR/dataset/pytdc_new/"
train1_file = base_path + "train1_PyTDC.csv"
train2_file = base_path + "train2_PyTDC.csv"
test_file = base_path + "test_PyTDC.csv"

# Step 4: Load the CSV files
train1_df = pd.read_csv(train1_file)
train2_df = pd.read_csv(train2_file)
test_df = pd.read_csv(test_file)

# Step 5: Merge the train datasets
merged_train_df = pd.concat([train1_df, train2_df], ignore_index=True)

# Step 6: Create transformation function for AVIB Beta format
def transform_to_avib_format(df):
    avib_df = pd.DataFrame({
        "tcra": "unknown",  # Set the 'tcra' column to 'unknown'
        "tcrb": df["tcr"],  # Map 'tcr' to 'tcrb'
        "peptide": df["epitope_aa"],  # Map 'epitope_aa' to 'peptide'
        "sign": df["label"],  # Map 'label' to 'sign'
    })
    return avib_df

# Step 7: Transform train and test datasets to AVIB Beta format
train_avib_df = transform_to_avib_format(merged_train_df)
test_avib_df = transform_to_avib_format(test_df)

# Step 8: Save the transformed datasets to CSV files
train_avib_path = "/content/train_PyTDC_to_AVIB.csv"
test_avib_path = "/content/test_PyTDC_to_AVIB.csv"
train_avib_df.to_csv(train_avib_path, index=False)
test_avib_df.to_csv(test_avib_path, index=False)

print(f"Transformed train file saved at: {train_avib_path}")
print(f"Transformed test file saved at: {test_avib_path}")

# Step 9: Verify the transformed files
print("Transformed Train Dataset Preview:")
print(train_avib_df)

print("Transformed Test Dataset Preview:")
print(test_avib_df)

Cloning into '11785-TCR'...
remote: Enumerating objects: 1135, done.
remote: Counting objects: 100% (345/345), done.
remote: Compressing objects: 100% (174/174), done.
remote: Total 1135 (delta 204), reused 298 (delta 157), pack-reused 790 (from 1)
Receiving objects: 100% (1135/1135), 4.70 MiB | 13.57 MiB/s, done.
Resolving deltas: 100% (640/640), done.
Transformed train file saved at: /content/train_PyTDC_to_AVIB.csv
Transformed test file saved at: /content/test_PyTDC_to_AVIB.csv
Transformed Train Dataset Preview:
          tcra                 tcrb      peptide  sign
0      unknown       CSVWGTGKTYEQYF     FLKEKGGL     1
1      unknown     CSATILAGVPYGEQYF     FLKEKGGL     1
2      unknown       CSASEGTSSYEQYF     FLKEKGGL     1
3      unknown  CAISESGYGGPPGANVLTF     FLKEKGGL     1
4      unknown  CAISEPGYRGPPGANVLTF     FLKEKGGL     1
...        ...                  ...          ...   ...
42459  unknown       CASSLEWGGETQYF    RLDKVEAEV     0
42460  unknown         CASSAREGKLFF    

Attentive Variational Information Bottleneck.

In this notebook, we train and test the Attentive Variational Information Bottleneck (MVIB [1] with Attention of Experts) and MVIB on all datasets.

In [2]:
# https://github.com/nec-research/vibtcr/blob/07169bf6a0a0f1620b56f9d744d0e8bfee50d38d/notebooks/notebooks.classification/avib.ipynb#L48

!git clone https://github.com/nec-research/vibtcr.git
# import sys
# sys.path.append('/content/vibtcr/vibtcr')
import os
os.chdir('/content/vibtcr/vibtcr')
current_path = os.getcwd()
os.listdir(current_path)

# !pip install numpy pandas scikit-learn torch torchvision torchaudio tqdm

# !pip install numpy==1.19.4
# !pip install pandas==1.1.4
# !pip install scikit-learn==0.24.2
# !pip install torch==1.10.0 torchvision==0.11.1 torchaudio==0.10.0 tqdm

Cloning into 'vibtcr'...
remote: Enumerating objects: 219, done.
remote: Counting objects: 100% (39/39), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 219 (delta 19), reused 13 (delta 4), pack-reused 180 (from 1)
Receiving objects: 100% (219/219), 44.30 MiB | 21.47 MiB/s, done.
Resolving deltas: 100% (93/93), done.


['vibtcr', 'requirements.txt', 'setup.py']

In [3]:
import pandas as pd
import torch
import numpy as np
import random
from sklearn.model_selection import train_test_split
from torch.utils.data.sampler import WeightedRandomSampler
from tqdm import tqdm
from sklearn.metrics import (
    roc_auc_score, accuracy_score, precision_score, recall_score, f1_score,
    precision_recall_curve, auc
)
from vibtcr.dataset import TCRDataset
from vibtcr.mvib.mvib import MVIB
from vibtcr.mvib.mvib_trainer import TrainerMVIB

# Step 1: Define helper functions
metrics = [
    'auROC',
    'Accuracy',
    'Recall',
    'Precision',
    'F1 score',
    'auPRC'
]

def pr_auc(y_true, y_prob):
    precision, recall, thresholds = precision_recall_curve(y_true, y_prob)
    pr_auc = auc(recall, precision)
    return pr_auc

def get_scores(y_true, y_prob, y_pred):
    """
    Compute a df with all classification metrics and respective scores.
    """
    scores = [
        roc_auc_score(y_true, y_prob),
        accuracy_score(y_true, y_pred),
        recall_score(y_true, y_pred),
        precision_score(y_true, y_pred),
        f1_score(y_true, y_pred),
        pr_auc(y_true, y_prob)
    ]
    df = pd.DataFrame(data={'score': scores, 'metrics': metrics})
    return df

def set_random_seed(random_seed):
    random.seed(random_seed)
    np.random.seed(random_seed)
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)

# Step 2: Model and Training Configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
batch_size = 4096
epochs = 500
lr = 1e-3
z_dim = 150
beta = 1e-6
early_stopper_patience = 50
monitor = 'auROC'
lr_scheduler_param = 10
joint_posterior = "aoe"
results_base = "/content/"

In [4]:
# Step 3: File paths
train_path = "/content/train_PyTDC_to_AVIB.csv"
test_path = "/content/test_PyTDC_to_AVIB.csv"

# Step 4: Load data
df_train_full = pd.read_csv(train_path)
df_test_full = pd.read_csv(test_path)

# Step 5: Train and Evaluate
for i in range(5):  # 5 independent train/test splits
    set_random_seed(i)

    # Split training and validation sets
    df_train, df_val = train_test_split(df_train_full, test_size=0.2, stratify=df_train_full.sign, random_state=i)

    # Create datasets and scalers
    scaler = TCRDataset(df_train.copy(), torch.device("cpu"), cdr3b_col='tcrb', cdr3a_col=None).scaler
    ds_test = TCRDataset(df_test_full, torch.device("cpu"), cdr3b_col='tcrb', cdr3a_col=None, scaler=scaler)
    ds_train = TCRDataset(df_train, device, cdr3b_col='tcrb', cdr3a_col=None, scaler=scaler)
    ds_val = TCRDataset(df_val, device, cdr3b_col='tcrb', cdr3a_col=None, scaler=scaler)

    # Train loader with balanced sampling
    class_count = np.array([df_train[df_train.sign == 0].shape[0], df_train[df_train.sign == 1].shape[0]])
    weight = 1. / class_count
    samples_weight = torch.tensor([weight[s] for s in df_train.sign])
    sampler = WeightedRandomSampler(samples_weight, len(samples_weight))
    train_loader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size, sampler=sampler)

    # Validation loader with balanced sampling
    class_count_val = np.array([df_val[df_val.sign == 0].shape[0], df_val[df_val.sign == 1].shape[0]])
    weight_val = 1. / class_count_val
    samples_weight_val = torch.tensor([weight_val[s] for s in df_val.sign])
    val_sampler = WeightedRandomSampler(samples_weight_val, len(samples_weight_val))
    val_loader = torch.utils.data.DataLoader(ds_val, batch_size=batch_size, sampler=val_sampler)

    # Initialize model and trainer
    model = MVIB(z_dim=z_dim, device=device, joint_posterior=joint_posterior).to(device)
    trainer = TrainerMVIB(
        model,
        epochs=epochs,
        lr=lr,
        beta=beta,
        checkpoint_dir=".",
        mode="bimodal",
        lr_scheduler_param=lr_scheduler_param
    )

    # Train the model
    checkpoint = trainer.train(train_loader, val_loader, early_stopper_patience, monitor)

    # Test the model
    model = MVIB.from_checkpoint(checkpoint, torch.device("cpu"))
    pred = model.classify(pep=ds_test.pep, cdr3b=ds_test.cdr3b, cdr3a=None)
    pred = pred.detach().numpy()
    df_test_full['prediction_' + str(i)] = pred.squeeze().tolist()

    # Save results
    output_file = results_base + f"mvib_bimodal_{joint_posterior}_split_{i}.csv"
    df_test_full.to_csv(output_file, index=False)
    print(f"Results saved at: {output_file}")

[VAL] Best epoch 16 | Best val score -0.760658 | DKL-prior 0.000219 | BCE 0.582420 | auROC 0.7607:  13%|█▎        | 65/500 [01:52<12:33,  1.73s/it]


Results saved at: /content/mvib_bimodal_aoe_split_0.csv


[VAL] Best epoch 16 | Best val score -0.756093 | DKL-prior 0.000256 | BCE 0.585861 | auROC 0.7561:  13%|█▎        | 65/500 [01:51<12:26,  1.72s/it]


Results saved at: /content/mvib_bimodal_aoe_split_1.csv


[VAL] Best epoch 40 | Best val score -0.762572 | DKL-prior 0.000265 | BCE 0.583028 | auROC 0.7626:  18%|█▊        | 89/500 [02:32<11:42,  1.71s/it]


Results saved at: /content/mvib_bimodal_aoe_split_2.csv


[VAL] Best epoch 8 | Best val score -0.752527 | DKL-prior 0.000225 | BCE 0.589948 | auROC 0.7525:  11%|█▏        | 57/500 [01:38<12:45,  1.73s/it]


Results saved at: /content/mvib_bimodal_aoe_split_3.csv


[VAL] Best epoch 38 | Best val score -0.772351 | DKL-prior 0.000271 | BCE 0.579199 | auROC 0.7724:  17%|█▋        | 87/500 [02:28<11:46,  1.71s/it]


Results saved at: /content/mvib_bimodal_aoe_split_4.csv


In [10]:
# Make a new CSV file to have the mean prediction value
file_path = "/content/mvib_bimodal_aoe_split_4.csv"
df = pd.read_csv(file_path)
df['prediction_mean'] = df[[f'prediction_{i}' for i in range(5)]].mean(axis=1)
output_file = "/content/mvib_bimodal_aoe_mean.csv"
df.to_csv(output_file, index=False)
print(df)

         tcra               tcrb     peptide  sign  prediction_0  \
0     unknown  CASSEWGMDGTTDTQYF   TLIGDCATV     1      0.601226   
1     unknown       CASGQDTGELFF   FIAGLIAIV     1      0.548987   
2     unknown     CSVSGNPSTGELFF   KLSYGIATV     1      0.873805   
3     unknown  CASSFHSGVPMGETQYF   ALSKGVHFV     1      0.619617   
4     unknown  CASSASSVQLLGDTQYF   RLRAEAQVK     1      0.596857   
...       ...                ...         ...   ...           ...   
4713  unknown     CASSIGQGARGYTF  ELAGIGILTV     1      0.876717   
4714  unknown   CASSDRGGRNTDTQYF   LVLSVNPYV     0      0.434205   
4715  unknown    CASSYGQGPAGEAFF   IQYIDIGNY     1      0.576165   
4716  unknown    CASSDREVDYNEQFF   KLWAQCVQL     1      0.763381   
4717  unknown        CASSEDAGYTF   KLWAQCVQL     1      0.736568   

      prediction_1  prediction_2  prediction_3  prediction_4  prediction_mean  
0         0.838134      0.650785      0.462484      0.645073         0.639541  
1         0.554488     

In [2]:
# reference code
# https://github.com/Yichuan0712/11785-TCR/blob/main/xgb.py

# !pip install -q condacolab
# import condacolab
# condacolab.install()
# !conda install -c conda-forge rdkit -y

!pip install rdkit-pypi
import pandas as pd
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, average_precision_score, f1_score
from sklearn.preprocessing import OneHotEncoder
from rdkit import Chem
from rdkit.Chem import Descriptors

# Function to extract features from SMILES strings
def extract_features(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return pd.Series([None] * 5)
    features = {
        'mol_weight': Descriptors.MolWt(mol),
        'logP': Descriptors.MolLogP(mol),
        'tpsa': Descriptors.TPSA(mol),
        'num_h_donors': Descriptors.NumHDonors(mol),
        'num_h_acceptors': Descriptors.NumHAcceptors(mol)
    }
    return pd.Series(features)

# Function to prepare features for the dataset
def prepare_features(df, use_smi=False):
    if use_smi and 'epitope_smi' in df.columns:
        smiles_features = df['epitope_smi'].apply(extract_features)
        df = pd.concat([df, smiles_features], axis=1)
        df = df.drop(columns=['epitope_smi'])
    return df

# Load the dataset
file_path = "/content/mvib_bimodal_aoe_mean.csv"
df = pd.read_csv(file_path)

# Split features and labels
X = df[['tcrb', 'peptide', 'prediction_0']]
y_true = df['sign']

# Prepare features (you can expand this based on additional requirements)
X = prepare_features(X, use_smi=False)

# Evaluate model metrics
y_pred = (df['prediction_0'] >= 0.5).astype(int)
y_pred_proba = df['prediction_0']

# Calculate metrics
accuracy = accuracy_score(y_true, y_pred)
auroc = roc_auc_score(y_true, y_pred_proba)
aupr = average_precision_score(y_true, y_pred_proba)
f1 = f1_score(y_true, y_pred)

# Print the evaluation results
print(f'Accuracy: {accuracy:.2f}')
print(f'AUROC: {auroc:.2f}')
print(f'AUPR: {aupr:.2f}')
print(f'F1 Score: {f1:.2f}')
print('Classification Report:')
print(classification_report(y_true, y_pred))

# compare with https://github.com/Yichuan0712/11785-TCR/blob/b1a82e928d5e753c746f82e3d552bd7e1ed9a4ff/colab/11785_project_runner.ipynb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 69.2 MB/s eta 0:00:00
Accuracy: 0.64
AUROC: 0.70
AUPR: 0.68
F1 Score: 0.65
Classification Report:
              precision    recall  f1-score   support

           0       0.64      0.63      0.64      2335
           1       0.64      0.66      0.65      2383

    accuracy                           0.64      4718
   macro avg       0.64      0.64      0.64      4718
weighted avg       0.64      0.64      0.64      4718

